In [4]:
import os
import json
import pandas as pd
from sklearn.preprocessing import RobustScaler
from io import BytesIO
import subprocess
import numpy as np
# Now 'ecg_classifier' contains your loaded model
import os

from tensorflow.keras.models import load_model

In [11]:
model_path = r"C:\Users\USER\Desktop\h5\ecggrad (1).h5" # Raw string notation for Windows paths
model = load_model(model_path)
def predict(image_path):
    command = f'plotdigitizer "{image_path}" -p 0,0 -p 2,0 -p 0,1 -l 2,29 -l 4,5 -l 22,5'
    output = subprocess.run(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    print("STDOUT:", output.stdout)
    print("STDERR:", output.stderr)

    if output.returncode != 0:
        raise RuntimeError("plotdigitizer failed to run correctly")

    image_path2 = os.path.splitext(image_path)[0] + ".jpg.traj.csv"
    if not os.path.exists(image_path2):
        raise FileNotFoundError(f"Expected file not found: {image_path2}")

    df = pd.read_csv(image_path2, sep=' ', header=None)
    scaler = RobustScaler()
    df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
    df = df.iloc[:, 1:2]

    df = df.head(10).T.reset_index(drop=True)

    df.columns = ['0_pre-RR', '0_pPeak', '0_rPeak', '0_sPeak', '0_qt_interval',
                  '1_pre-RR', '1_qPeak', '1_qt_interval', '1_qrs_morph0', '1_qrs_morph1']

    df = df.apply(pd.to_numeric, errors='coerce')

    # Make predictions using the loaded model
    predictions = model.predict(df.values) 

    # Parse predictions
    predicted_label_index = np.argmax(predictions)
    class_labels = ["N", "VEB", "SVEB", "Fusion"]
    predicted_label = class_labels[predicted_label_index]
    print(f"The predicted label is: {predicted_label}")


In [12]:
predict(r"C:\Users\USER\Desktop\veb\veb.jpg")

STDOUT: 
STDERR: INFO:root:Extracting trajectories from C:\Users\USER\Desktop\veb\veb.jpg
INFO:root:[(2, 29), (4, 5), (22, 5)] â†’ origin 2, 5
INFO:root: 236.6016922361692  36.560834167149366
INFO:root:Extracting trajectory for color 0
INFO:root:Wrote trajectory to C:\Users\USER\Desktop\veb\veb.jpg.traj.csv

1/1 [==============================] - 1s 578ms/step
The predicted label is: VEB
